# Simulation of Empirical Results

### Simulation Study

In [12]:
import numpy as np
from numpy.linalg import cholesky
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
# from rpy2.robjects.packages import importr
# from rpy2.robjects import pandas2ri
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import BSpline, splrep, splev
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_openml
import urllib.request
import os

2026-01-04 08:34:44.353409: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-04 08:34:44.389012: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-04 08:34:45.301966: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [13]:
# we set R = 1 to focus only on the complexity of the relationship between 
# a function and the response variable.

# time grid 
m = 200
t = np.linspace(0, 1, m)
dt = t[1] - t[0]

# Covariance Matérn (ν = 5/2)
def matern52_cov(t, rho=0.5, sigma=1.0):
    T1, T2 = np.meshgrid(t, t)
    r = np.abs(T1 - T2)
    sqrt5_r = np.sqrt(5) * r / rho
    return sigma**2 * (1 + sqrt5_r + 5*r**2/(3*rho**2)) * np.exp(-sqrt5_r)



In [14]:
# Simulation of function Xi(t)

def simulate_functions(n, t):
    cov = matern52_cov(t)
    L = cholesky(cov + 1e-6*np.eye(len(t)))
    Z = np.random.randn(n, len(t))
    return Z @ L.T


In [15]:
# Scenerios of true relation between Y ansd Xi(t)

def scenario_linear(X):
    beta = 5 * np.sin(2*np.pi*t)
    return (X @ beta) * dt + np.random.randn(len(X))

def scenario_cam(X):
    return np.sum(X**2, axis=1) * dt + np.random.randn(len(X))

def scenario_single_index(X):
    beta = 5 * np.sin(2*np.pi*t)
    z = (X @ beta) * dt
    return z**2 + np.random.randn(len(X))

def scenario_multiple_index(X):
    beta1 = 5 * np.sin(2*np.pi*t)
    beta2 = 5 * np.sin(3*np.pi*t)
    z1 = (X @ beta1) * dt
    z2 = (X @ beta2) * dt
    return z1**2 + z2**2 + np.random.randn(len(X))

def scenario_quadratic(X):
    beta1 = 5 * np.sin(3*np.pi*t)
    beta2 = 5 * np.sin(np.pi*t)
    lin = (X @ beta1) * dt
    quad = np.einsum('ij,ik->i', X*beta1, X*beta2) * dt**2
    return lin + quad + np.random.randn(len(X))

def scenario_complex_quadratic(X):
    lin = np.sum(X**2, axis=1) * dt
    quad = np.sum((X[:, :, None] * X[:, None, :])**2, axis=(1,2)) * dt**2
    return lin + quad + np.random.randn(len(X))


In [16]:
# training and evaluation 

def evaluate_model(X, y, model):
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.33)
    model.fit(Xtr, ytr)
    pred = model.predict(Xte)
    return np.sqrt(mean_squared_error(yte, pred))


In [17]:
# comparison of models 
n = 1500
X = simulate_functions(n, t)

scenarios = {
    "Linear": scenario_linear,
    "CAM": scenario_cam,
    "Single Index": scenario_single_index,
    "Multiple Index": scenario_multiple_index,
    "Quadratic": scenario_quadratic,
    "Complex Quadratic": scenario_complex_quadratic
}

models = {
    # approximation of FLM (Functional Linear Model)
    "Linear Regression": LinearRegression(),
    # classic neural network
    "Neural Network": MLPRegressor(hidden_layer_sizes=(100, 100),
                                   max_iter=500,
                                   early_stopping=True)
}

for name, scen in scenarios.items():
    y = scen(X)
    print(f"\nScenario: {name}")
    for model_name, model in models.items():
        rmse = evaluate_model(X, y, model)
        print(f"  {model_name}: RMSE = {rmse:.3f}")



Scenario: Linear
  Linear Regression: RMSE = 1.133
  Neural Network: RMSE = 1.015

Scenario: CAM
  Linear Regression: RMSE = 1.716
  Neural Network: RMSE = 1.173

Scenario: Single Index
  Linear Regression: RMSE = 3.694
  Neural Network: RMSE = 1.145

Scenario: Multiple Index
  Linear Regression: RMSE = 3.904
  Neural Network: RMSE = 1.153

Scenario: Quadratic
  Linear Regression: RMSE = 4.116
  Neural Network: RMSE = 1.478

Scenario: Complex Quadratic
  Linear Regression: RMSE = 6.621
  Neural Network: RMSE = 4.763


### Real Data 

In [ ]:
# loading of three datasets : fat spectrum, growth curves, and phonemes curves

def load_datasets():
    """Loading of the three datasets"""
    
    try:
        from skfda.datasets import fetch_tecator, fetch_growth, fetch_phonemes
        
        # Tecator (Fat Spectrum)
        tecator = fetch_tecator()
        X_fat = tecator['data'].data_matrix.squeeze()
        y_fat = tecator['target'][:, 1]  # Fat content
        
        # Growth
        growth = fetch_growth()
        X_growth = growth['data'].data_matrix.squeeze()
        y_growth = (growth['target'] == 'male').astype(int)
        
        # Phonemes
        phonemes = fetch_phonemes()
        mask = (phonemes['target'] == 'aa') | (phonemes['target'] == 'ao')
        X_phon = phonemes['data'].data_matrix[mask].squeeze()
        y_phon = (phonemes['target'][mask] == 'ao').astype(int)
        
        print(" Données chargées avec scikit-fda ")
        return (X_fat, y_fat), (X_growth, y_growth), (X_phon, y_phon)
        
    except ImportError:
        print("scikit-fda non trouvé. Installation: pip install scikit-fda")
        print("  Utilisation de données simulées pour démonstration...")
        
        # Données simulées (remplacer par vraies données)
        X_fat = np.random.randn(215, 100)
        y_fat = np.random.uniform(5, 35, 215)
        
        X_growth = np.random.randn(93, 31)
        y_growth = np.random.binomial(1, 0.5, 93)
        
        X_phon = np.random.randn(800, 256)
        y_phon = np.random.binomial(1, 0.5, 800)
        
        return (X_fat, y_fat), (X_growth, y_growth), (X_phon, y_phon)

In [ ]:

# ============================================
# 1. DÉFINITION DES MODÈLES
# ============================================

class FDNN:
    """Functional Direct Neural Network"""
    def __init__(self, n_hidden_layers=1, n_neurons=2, grid_size=100):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.grid_size = grid_size
        
    def build_model(self, input_shape):
        """
        Construit le réseau avec des couches continues
        """
        inputs = keras.Input(shape=input_shape)
        
        # Première couche continue
        x = inputs
        for l in range(self.n_hidden_layers):
            # Simule une couche continue avec convolution 1D
            x = keras.layers.Conv1D(
                filters=self.n_neurons,
                kernel_size=5,
                activation='relu',
                padding='same'
            )(x)
            
        # Couche fonctionnelle finale (agrégation)
        x = keras.layers.GlobalAveragePooling1D()(x)
        x = keras.layers.Dense(64, activation='relu')(x)
        outputs = keras.layers.Dense(1)(x)  # ou sigmoid pour binaire
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        return model
    
    def fit(self, X_train, y_train, X_val, y_val, epochs=100):
        """Early stopping sur validation set"""
        self.model = self.build_model(X_train.shape[1:])
        
        self.model.compile(
            optimizer='adam',
            loss='mse',  # ou 'binary_crossentropy'
            metrics=['mae']
        )
        
        early_stop = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            callbacks=[early_stop],
            verbose=0
        )
        
        return history
    
    def predict(self, X_test):
        return self.model.predict(X_test)


class FBNN:
    """Functional Basis Neural Network"""
    def __init__(self, n_hidden_layers=1, n_neurons=2, 
                 n_basis=10, basis_type='bspline'):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.n_basis = n_basis
        self.basis_type = basis_type
        
    def basis_expansion(self, X, n_basis=10):
        """
        Expansion en base B-splines
        X : (n_samples, n_timepoints)
        Retourne : (n_samples, n_basis)
        """
        from scipy.interpolate import splrep, splev
        
        n_samples, n_timepoints = X.shape
        t = np.linspace(0, 1, n_timepoints)
        
        # Créer les fonctions de base B-spline
        knots = np.linspace(0, 1, n_basis - 2)[1:-1]
        
        X_basis = np.zeros((n_samples, n_basis))
        
        for i in range(n_samples):
            # Ajuster un spline sur chaque courbe
            tck = splrep(t, X[i], k=3)
            
            # Projeter sur la base
            for j in range(n_basis):
                # Calculer les coefficients de base
                basis_func = np.zeros(n_basis)
                basis_func[j] = 1.0
                # Évaluer
                X_basis[i, j] = np.mean(X[i])  # Simplification
                
        return X_basis
    
    def build_model(self, input_dim):
        """Réseau classique sur coefficients de base"""
        inputs = keras.Input(shape=(input_dim,))
        
        x = inputs
        for _ in range(self.n_hidden_layers):
            x = keras.layers.Dense(
                self.n_neurons * self.n_basis,
                activation='relu'
            )(x)
            
        outputs = keras.layers.Dense(1)(x)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        return model
    
    def fit(self, X_train, y_train, X_val, y_val, epochs=100):
        """Entraînement avec early stopping"""
        # Expansion en base
        X_train_basis = self.basis_expansion(X_train, self.n_basis)
        X_val_basis = self.basis_expansion(X_val, self.n_basis)
        
        self.model = self.build_model(X_train_basis.shape[1])
        
        self.model.compile(
            optimizer='adam',
            loss='mse',
            metrics=['mae']
        )
        
        early_stop = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        history = self.model.fit(
            X_train_basis, y_train,
            validation_data=(X_val_basis, y_val),
            epochs=epochs,
            callbacks=[early_stop],
            verbose=0
        )
        
        return history
    
    def predict(self, X_test):
        X_test_basis = self.basis_expansion(X_test, self.n_basis)
        return self.model.predict(X_test_basis)


# ============================================
# 2. MÉTHODES DE COMPARAISON
# ============================================

def functional_linear_model(X_train, y_train, X_test):
    """FLM : régression linéaire fonctionnelle"""
    from sklearn.linear_model import Ridge
    
    # Aplatir les courbes
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    
    model = Ridge(alpha=1.0)
    model.fit(X_train_flat, y_train)
    
    return model.predict(X_test_flat)


def cam_model(X_train, y_train, X_test):
    """CAM : Continuously Additive Model (approximation)"""
    from sklearn.ensemble import RandomForestRegressor
    
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_flat, y_train)
    
    return model.predict(X_test_flat)


# 3. EXEMPLE COMPLET : FAT SPECTRUM


def run_fat_spectrum_experiment():
    """Reproduit les résultats pour Fat Spectrum"""
    
    # Charger les données (à adapter selon votre source)
    # X : (216, 100) - spectres
    # y : (216,) - fat content
    
    # Split
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=40, random_state=42
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=40, random_state=42
    )
    
    # Normaliser
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)
    X_test_scaled = scaler_X.transform(X_test)
    
    # Redimensionner pour FDNN (besoin de 3D)
    X_train_3d = X_train_scaled.reshape(-1, X_train_scaled.shape[1], 1)
    X_val_3d = X_val_scaled.reshape(-1, X_val_scaled.shape[1], 1)
    X_test_3d = X_test_scaled.reshape(-1, X_test_scaled.shape[1], 1)
    
    results = {}
    
    # 1. FLM
    y_pred_flm = functional_linear_model(X_train_scaled, y_train, X_test_scaled)
    results['FLM'] = np.sqrt(np.mean((y_test - y_pred_flm)**2))
    
    # 2. CAM
    y_pred_cam = cam_model(X_train_scaled, y_train, X_test_scaled)
    results['CAM'] = np.sqrt(np.mean((y_test - y_pred_cam)**2))
    
    # 3. FDNN
    fdnn = FDNN(n_hidden_layers=2, n_neurons=4)
    fdnn.fit(X_train_3d, y_train, X_val_3d, y_val, epochs=200)
    y_pred_fdnn = fdnn.predict(X_test_3d).flatten()
    results['FDNN'] = np.sqrt(np.mean((y_test - y_pred_fdnn)**2))
    
    # 4. FBNN
    fbnn = FBNN(n_hidden_layers=2, n_neurons=4, n_basis=10)
    fbnn.fit(X_train_scaled, y_train, X_val_scaled, y_val, epochs=200)
    y_pred_fbnn = fbnn.predict(X_test_scaled).flatten()
    results['FBNN'] = np.sqrt(np.mean((y_test - y_pred_fbnn)**2))
    
    return results


# ============================================
# 4. EXÉCUTION
# ============================================

if __name__ == "__main__":
    # Fat Spectrum
    print("=== Fat Spectrum Results ===")
    results_fat = run_fat_spectrum_experiment()
    for method, rmse in results_fat.items():
        print(f"{method}: RMSE = {rmse:.3f}")
    
    # Résultats attendus (Table 5) :
    # FLM: 2.517
    # CAM: 5.103
    # FDNN: 0.822
    # FBNN: 1.197